# Get all codes using OLC Decode

In [1]:
from openlocationcode import openlocationcode as olc
import pluscodeplus
import shapely
import folium
import geopandas as gpd
import numpy as np
import os
from pathlib import Path
import pandas as pd
from shapely.geometry import Polygon
from glob import glob

In [2]:
shapely.__version__

'1.8.5.post1'

In [3]:
gpd.__version__

'0.12.1'

In [4]:
list_file_folder = Path("/Users/abharathi/Documents/gis_data/pluscodes/pc_name_lists")
usa_pc8_list = list_file_folder / "usa_pc8_list.csv"

In [5]:
pc8_list_df = pd.read_csv(usa_pc8_list)
pc8_list_df.shape

(20549278, 1)

In [6]:
pc8_list_df.head()

,PLUSCODE8
0,858WMW7W
1,858W659R
2,858WW398
3,858WH9Q8
4,858WQ7C5


In [7]:
print(pc8_list_df['PLUSCODE8'].loc[0])
dc = olc.decode(f"{pc8_list_df['PLUSCODE8'].loc[0]}+")
type(dc)

858WMW7W


openlocationcode.openlocationcode.CodeArea

In [8]:
dc.codeLength

8

In [9]:
dc.latlng() # center coords

[36.66375, -101.05375000000001]

In [10]:
# xmin, xmax; ymin, ymax
dc.longitudeLo, dc.longitudeHi, dc.latitudeLo, dc.latitudeHi

(-101.055, -101.05250000000001, 36.6625, 36.665)

In [11]:
type(pluscodeplus.get_pluscode_geometry([f"{pc8_list_df['PLUSCODE8'].loc[0]}+"]))


geopandas.geodataframe.GeoDataFrame

## Iterate and produce pc8 DF

In [12]:
pc8_list_plus = list(pc8_list_df['PLUSCODE8'])
pc8_list_plus = [f"{e}+" for e in pc8_list_plus]

In [13]:
pc8_list_plus[0]

'858WMW7W+'

In [14]:
%%time
pc8_gdf = pluscodeplus.get_pluscode_geometry(pc8_list_plus)

CPU times: user 7min 38s, sys: 44.6 s, total: 8min 22s
Wall time: 8min 44s


In [15]:
pc8_gdf.shape

(20549278, 2)

In [16]:
pc8_gdf.head()

,pluscode,geometry
0,858WMW7W+,"POLYGON ((-101.05250 36.66250, -101.05250 36.6..."
1,858W659R+,"POLYGON ((-101.80750 36.21750, -101.80750 36.2..."
2,858WW398+,"POLYGON ((-101.93250 36.91750, -101.93250 36.9..."
3,858WH9Q8+,"POLYGON ((-101.63250 36.58750, -101.63250 36.5..."
4,858WQ7C5+,"POLYGON ((-101.74000 36.77000, -101.74000 36.7..."


In [17]:
list_file_folder.parent / "data_files" / "usa_pc8.geojson"

PosixPath('/Users/abharathi/Documents/gis_data/pluscodes/data_files/usa_pc8.geojson')

In [19]:
%%time
pc8_gdf.to_parquet(path=list_file_folder.parent / "usa_pc8_geo.parquet",
                   index=False, compression="snappy")

CPU times: user 8min 28s, sys: 21.4 s, total: 8min 49s
Wall time: 9min 18s


## Make grid for Japan

In [23]:
jp_file_list = {"0000+": list_file_folder / "jp_pc4_list.csv",
                           "00+": list_file_folder / "jp_pc6_list.csv",
                           "+": list_file_folder / "jp_pc8_list.csv"}

In [24]:
for prefix, file in jp_file_list.items():
    print(f"processing {file}")
    df = pd.read_csv(str(file))
    pc_list_plus = list(df['PLUSCODE'])
    pc_list_plus = [f"{e}{prefix}" for e in pc_list_plus]
    
    print("read file into df")
    gdf = pluscodeplus.get_pluscode_geometry(pc_list_plus)
    print("made gdf")
    
    file_name = f'{file.name.split(".csv")[0].split("_list")[0]}_geo.parquet'
    gdf.to_parquet(path=list_file_folder.parent / file_name,
                   index=False, compression="snappy")
    print(f"wrote {file_name} to disk")

processing /Users/abharathi/Documents/gis_data/pluscodes/pc_name_lists/jp_pc4_list.csv
read file into df
made gdf
wrote jp_pc4_geo.parquet to disk
processing /Users/abharathi/Documents/gis_data/pluscodes/pc_name_lists/jp_pc6_list.csv
read file into df
made gdf
wrote jp_pc6_geo.parquet to disk
processing /Users/abharathi/Documents/gis_data/pluscodes/pc_name_lists/jp_pc8_list.csv
read file into df
made gdf
wrote jp_pc8_geo.parquet to disk


In [25]:
gdf.shape

(1495167, 2)